In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
results_df_dict = {}

for root, dirs, files in os.walk("results/"):
    for file in sorted(files):
        file_path = f"{root}{file}"
        record_name = file.split('.')[2]
        tmp_df = pd.read_parquet(file_path)
        results_df_dict[record_name] = tmp_df

In [3]:
keys = list(results_df_dict.keys())
keys

['2018_01_01_desktop',
 '2019_02_01_desktop',
 '2020_01_01_desktop',
 '2021_01_01_desktop',
 '2022_01_01_desktop',
 '2023_01_01_desktop']

In [4]:
# nel_include_subdomains default false
# nel_success_fraction default 0.0
# nel_failure_fraction default 1.0

from operator import is_

for key in keys[1:]:
    tmp_df = results_df_dict[key]

    tmp_df['nel_include_subdomains'] = tmp_df['nel_include_subdomains'].astype('bool')
    tmp_df['nel_success_fraction'] = tmp_df['nel_success_fraction'].astype('float')
    tmp_df['nel_failure_fraction'] = tmp_df['nel_failure_fraction'].astype('float')

    tmp_df['nel_include_subdomains'] = np.where(np.vectorize(is_)(tmp_df['nel_include_subdomains'], None), False, tmp_df['nel_include_subdomains'])
    tmp_df['nel_success_fraction'] = np.where(np.vectorize(is_)(tmp_df['nel_success_fraction'], None), 0.0, tmp_df['nel_success_fraction'])
    tmp_df['nel_failure_fraction'] = np.where(np.vectorize(is_)(tmp_df['nel_failure_fraction'], None), 1.0, tmp_df['nel_failure_fraction'])

In [5]:
def get_virtue(results_df_dict, what, col_name):
    virtue_df = pd.DataFrame()

    for key in keys[1:]:
        tmp_df = results_df_dict[key][[what, 'requestid']].groupby(by=[what]).count().sort_values(by='requestid', ascending=False).reset_index()
        year = key.split('_')[0]
        tmp_df.columns = [f'{year}_{col_name}', f'{year}_cnt']
        tmp_df[f'{year}_prcnt'] = 100/len(results_df_dict[key])*tmp_df[f'{year}_cnt']
        virtue_df = pd.merge(virtue_df, tmp_df, how='outer', left_index=True, right_index=True)

    return virtue_df

In [6]:
success_df = get_virtue(results_df_dict, 'nel_success_fraction', 'success')
success_df

,2019_success,2019_cnt,2019_prcnt,2020_success,2020_cnt,2020_prcnt,2021_success,2021_cnt,2021_prcnt,2022_success,2022_cnt,2022_prcnt,2023_success,2023_cnt,2023_prcnt
0,0.0000,23.0,6.478873,0.00010,79448.0,98.989521,0.00010,184574.0,23.455541,0.00000,642181,91.058634,0.00000,1181443,67.129808
1,0.0100,2.0,0.563380,0.00066,31.0,0.038625,0.00000,728.0,0.092514,0.01000,50457,7.154596,0.01000,539896,30.676990
2,0.2500,2.0,0.563380,0.00000,27.0,0.033641,0.00100,176.0,0.022366,0.00100,198,0.028076,0.00100,9031,0.513143
3,0.0001,1.0,0.281690,0.00100,25.0,0.031149,0.00066,39.0,0.004956,0.00500,70,0.009926,0.00500,573,0.032558
4,NaN,NaN,NaN,0.05000,5.0,0.006230,0.01000,19.0,0.002415,0.00066,48,0.006806,0.20000,510,0.028978
5,NaN,NaN,NaN,0.25000,2.0,0.002492,1.00000,4.0,0.000508,0.00010,24,0.003403,0.00066,18,0.001023
6,NaN,NaN,NaN,0.01000,1.0,0.001246,0.05000,3.0,0.000381,0.05000,12,0.001702,1.00000,17,0.000966
7,NaN,NaN,NaN,0.10000,1.0,0.001246,0.10000,2.0,0.000254,0.20000,9,0.001276,0.05000,16,0.000909
8,NaN,NaN,NaN,1.00000,1.0,0.001246,NaN,NaN,NaN,0.10000,2,0.000284,0.00010,11,0.000625
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50000,2,0.000284,0.10000,10,0.000568


In [7]:
failure_df = get_virtue(results_df_dict, 'nel_failure_fraction', 'failure')
failure_df

,2019_failure,2019_cnt,2019_prcnt,2020_failure,2020_cnt,2020_prcnt,2021_failure,2021_cnt,2021_prcnt,2022_failure,2022_cnt,2022_prcnt,2023_failure,2023_cnt,2023_prcnt
0,0.00001,223.0,62.816901,0.01000,79464.0,99.009457,0.05000,634.0,0.080568,0.10000,10415.0,1.476804,0.100000,15254,0.866735
1,1.00000,33.0,9.295775,0.00001,268.0,0.333919,0.00001,325.0,0.041301,0.05000,695.0,0.098548,1.000000,9741,0.553485
2,0.10000,3.0,0.845070,1.00000,118.0,0.147024,1.00000,265.0,0.033676,1.00000,414.0,0.058704,0.001000,3319,0.188586
3,0.50000,2.0,0.563380,0.10000,14.0,0.017444,0.10000,117.0,0.014868,0.00001,393.0,0.055726,0.050000,880,0.050002
4,0.00100,1.0,0.281690,0.50000,4.0,0.004984,0.01000,100.0,0.012708,0.01000,76.0,0.010776,0.000010,584,0.033183
5,0.01000,1.0,0.281690,0.00010,2.0,0.002492,0.25000,5.0,0.000635,0.00100,39.0,0.005530,0.010000,119,0.006762
6,0.80000,1.0,0.281690,0.00100,2.0,0.002492,0.00010,4.0,0.000508,0.15000,13.0,0.001843,0.200000,79,0.004489
7,NaN,NaN,NaN,0.20000,2.0,0.002492,0.20000,4.0,0.000508,0.50000,7.0,0.000993,0.500000,63,0.003580
8,NaN,NaN,NaN,0.00000,1.0,0.001246,0.50000,3.0,0.000381,0.25000,4.0,0.000567,0.000001,10,0.000568
9,NaN,NaN,NaN,0.05000,1.0,0.001246,0.00100,1.0,0.000127,0.90000,4.0,0.000567,0.250000,7,0.000398


In [9]:
subdomains_df = get_virtue(results_df_dict, 'nel_include_subdomains', 'subdomains')
subdomains_df

,2019_subdomains,2019_cnt,2019_prcnt,2020_subdomains,2020_cnt,2020_prcnt,2021_subdomains,2021_cnt,2021_prcnt,2022_subdomains,2022_cnt,2022_prcnt,2023_subdomains,2023_cnt,2023_prcnt
0,True,348,98.028169,False,79502,99.056804,False,785793,99.858052,False,703790,99.794538,False,1746510,99.237019
1,False,7,1.971831,True,757,0.943196,True,1117,0.141948,True,1449,0.205462,True,13428,0.762981


In [10]:
results_df_dict[keys[5]].columns

Index(['requestid', 'type', 'ext', 'firstReq', 'status', 'url',
       'unique_domain_count_before_filtration',
       'unique_domain_firstreq_count_before_filtration', 'contains_nel',
       'nel_max_age', 'nel_failure_fraction', 'nel_success_fraction',
       'nel_include_subdomains', 'nel_report_to_group',
       'nel_count_before_filtration', 'rt_group', 'rt_endpoints', 'rt_url',
       'rt_url_sld'],
      dtype='object')

In [12]:
tmp_df = results_df_dict[keys[5]][['nel_failure_fraction', 'rt_url_sld', 'requestid']].groupby(by=['rt_url_sld', 'nel_failure_fraction']).count().sort_values(by='requestid', ascending=False).reset_index()


In [13]:
tmp_df

,rt_url_sld,nel_failure_fraction,requestid
0,cafe24.com,0.1000,14985
1,office.net,1.0000,8127
2,cloudfront.net,0.0010,3318
3,nelreports.net,1.0000,1200
4,wikimedia.org,0.0500,759
...,...,...,...
74,movizor-info.ru,1.0000,1
75,report-uri.com,0.0001,1
76,planer.io,1.0000,1
77,raiffeisen.ch,0.2500,1
